## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os 
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [27]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Udachnyy,66.4167,112.4000,61.45,93,100,1.90,RU,overcast clouds
1,1,Vanavara,60.3400,102.2797,60.66,69,100,11.23,RU,overcast clouds
2,2,Sangueya,10.7000,-14.3667,77.14,89,100,4.88,GN,overcast clouds
3,3,Fortuna,40.5982,-124.1573,60.85,91,100,1.99,US,overcast clouds
4,4,Ponoka,52.6768,-113.5815,57.18,87,100,9.13,CA,overcast clouds


In [28]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip?  78
What is the maximum temperature you would like for your trip?  88


In [32]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
12,12,Victoria,22.2855,114.1577,84.38,85,100,31.00,HK,overcast clouds
25,25,Jamestown,42.0970,-79.2353,79.02,50,100,7.00,US,overcast clouds
34,34,Kapaa,22.0752,-159.3190,80.58,80,100,7.00,US,overcast clouds
45,45,Matara,5.9485,80.5353,79.93,82,8,15.52,LK,clear sky
55,55,Palmer,42.1584,-72.3287,86.50,60,38,8.97,US,scattered clouds
57,57,Puerto Ayora,-0.7393,-90.3518,78.76,86,62,8.01,EC,broken clouds
58,58,Athens,37.9795,23.7162,86.25,37,0,10.36,GR,clear sky
59,59,Hami,42.8000,93.4500,81.64,17,100,7.76,CN,overcast clouds
63,63,Arraial Do Cabo,-22.9661,-42.0278,78.76,69,0,19.57,BR,clear sky
73,73,Itoman,26.1247,127.6694,78.82,89,75,27.63,JP,mist


In [33]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                168
City                   168
Lat                    168
Lng                    168
Max Temp               168
Humidity               168
Cloudiness             168
Wind Speed             168
Country                168
Current Description    168
dtype: int64

In [37]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [38]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Victoria,HK,84.38,overcast clouds,22.2855,114.1577,
25,Jamestown,US,79.02,overcast clouds,42.0970,-79.2353,
34,Kapaa,US,80.58,overcast clouds,22.0752,-159.3190,
45,Matara,LK,79.93,clear sky,5.9485,80.5353,
55,Palmer,US,86.50,scattered clouds,42.1584,-72.3287,
57,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,
58,Athens,GR,86.25,clear sky,37.9795,23.7162,
59,Hami,CN,81.64,overcast clouds,42.8000,93.4500,
63,Arraial Do Cabo,BR,78.76,clear sky,-22.9661,-42.0278,
73,Itoman,JP,78.82,mist,26.1247,127.6694,


In [39]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Victoria,HK,84.38,overcast clouds,22.2855,114.1577,Mini Central
25,Jamestown,US,79.02,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
34,Kapaa,US,80.58,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
45,Matara,LK,79.93,clear sky,5.9485,80.5353,Amaloh Boutique Resort
55,Palmer,US,86.50,scattered clouds,42.1584,-72.3287,Wedgewood Motel
...,...,...,...,...,...,...,...
669,Ramgarh,IN,82.83,overcast clouds,23.6333,85.5167,HOTEL TREAT RESIDENCY
670,Frutal,BR,87.46,clear sky,-20.0247,-48.9406,Hotel Recanto Dos Ipês
674,Umm Lajj,SA,85.77,clear sky,25.0213,37.2685,Moon light Furnished Units
675,Alibag,IN,82.47,overcast clouds,18.6411,72.8792,"Radisson Blu Resort & Spa - Alibaug, India"


In [42]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [43]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Database.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [45]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))